In [1]:
import pandas as pd
import numpy as np
from scipy import linalg
import time
import itertools
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from sklearn.externals import joblib

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

In [2]:
tfidf  = joblib.load('models/TFIDF_model.joblib')
# svd    = joblib.load('models/SVD_model.joblib')
logreg = joblib.load('models/LogReg_model_noSVD.joblib')

In [3]:
dfr = pd.read_csv('data/AirbnbReviews.csv.gz', compression='gzip')
dfb = pd.read_csv('data/AirbnbListings.csv.gz', compression='gzip')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
list(dfr.columns)

['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

In [5]:
stop_words = set(stopwords.words('english'))
punct = list(string.punctuation)
ps = PorterStemmer()

def clean_text(text):
    if isinstance(text, str):
        for ch in punct:
            text = text.replace(ch, '')
        word_tokens = text.lower().split() 
        clean_text = [ps.stem(w) for w in word_tokens if not w in stop_words]
        ct = ' '.join(clean_text)
        return ct
    else:
        return None

In [6]:
dfr['clean_text'] = dfr['comments'].progress_apply(clean_text)

In [7]:
print(len(dfr))
dfr = dfr.dropna(subset=['clean_text'])
print(len(dfr))

610649
610238


In [8]:
start_time = time.time()
# est_sent = logreg.predict(svd.transform(tfidf.transform(dfr['clean_text'])))
est_sent = logreg.predict(tfidf.transform(dfr['clean_text']))
print('--- {0} seconds ---'.format( time.time() - start_time ))

--- 11.47213625907898 seconds ---


In [9]:
dfr['est_rating'] = est_sent

In [10]:
dfb_tmp = dfr.groupby('listing_id').mean()[['est_rating']]

In [11]:
dfb2 = dfb.merge(dfb_tmp, how='right', left_on='id', right_on='listing_id').copy()
len(dfb2)

18233

In [12]:
print(len(dfb2))
dfb2 = dfb2.dropna(subset=['review_scores_rating'])
print(len(dfb2))

18233
17979


In [13]:
dfb2['diff_rating'] = dfb2.apply(lambda x: abs(x.review_scores_rating - x.est_rating*100), axis = 1)

In [14]:
dfb2['diff_rating'].mean()

5.849717659512015

In [ ]:
list(dfb.columns)

In [ ]:
list(dfb.)